In [8]:
import random
import math
import copy
from collections import deque, defaultdict
import gurobipy as gp
from gurobipy import GRB


In [9]:
def read_top_instance(file_path):
    with open(file_path, 'r') as file:
        # Read the first three lines
        n_line = file.readline().strip()
        m_line = file.readline().strip()
        tmax_line = file.readline().strip()

        # Parse n N
        _, N = n_line.split()
        N = int(N)

        # Parse m P
        _, P = m_line.split()
        P = int(P)

        # Parse tmax Tmax
        _, Tmax = tmax_line.split()
        Tmax = float(Tmax)

        # Read the remaining lines for each node
        nodes = []
        for i in range(N):
            line = file.readline().strip()
            if line == '':
                continue  # Skip empty lines
            x_str, y_str, s_str = line.split()
            x = float(x_str)
            y = float(y_str)
            s = float(s_str)
            nodes.append({'id': i, 'x': x, 'y': y, 'score': s})

        return N, P, Tmax, nodes


In [10]:
# define problem instance. 
class ProblemInstance:
    def __init__(self, file_path):
        """
        Initialize the problem instance by reading data from a file.

        :param file_path: Path to the instance data file
        """
        self.N, self.P, self.Tmax, self.nodes = read_top_instance(file_path)
        self.distance_matrix = self.compute_distance_matrix()
        self.original_start_id = 0
        self.original_end_id = self.N - 1

    def compute_distance_matrix(self):
        """
        Compute the Euclidean distance matrix keyed by node IDs.

        :return: A dictionary of dictionaries representing the distance matrix, 
                where distance_matrix[id1][id2] = distance between node id1 and id2.
        """
        # Initialize the distance matrix as a nested dictionary
        distance_matrix = {}
        
        # Extract all node IDs and coordinates for convenience
        nodes_info = {node['id']: (node['x'], node['y']) for node in self.nodes}

        for id1, (x1, y1) in nodes_info.items():
            distance_matrix[id1] = {}
            for id2, (x2, y2) in nodes_info.items():
                if id1 == id2:
                    distance_matrix[id1][id2] = 0.0
                else:
                    distance = math.hypot(x1 - x2, y1 - y2)
                    distance_matrix[id1][id2] = distance

        return distance_matrix


In [11]:

import logging


class HALNS:
    def __init__(self, file_path, parameters):
        """
        Initialize the HALNS heuristic with the given problem instance file and parameters.

        :param file_path: Path to the TOP instance data file
        :param parameters: A dictionary containing algorithm parameters
        """
        self.problem = ProblemInstance(file_path)
        self.params = parameters

        # Initialize variables
        self.s_best = None
        self.s_adm = None
        self.T = self.params['T0']
        self.seg = 0
        self.iteration_best = 0
        self.s_best = None
        self.s_adm = None
        self.routes_list = []  # List of tuples: (route, profit, coverage)
        self.generated_routes = set()  # Set of frozensets representing unique routes


        # Initialize scores for strategies and operators
        # Mapped from the paper: dynamic profit/time, highest profit, random, LRFI
        self.selection_strategies = [
            'dynamic_profit_per_time', 
            'highest_profit',
            'random',
            'lrfi'
        ]

        self.removal_ops = [
            'random_removal',
            'lowest_profit_removal',
            'largest_saving_removal',
            'route_removal',
            'sequence_removal'
        ]
        
        self.insertion_ops = [
            'first_available',
            'last_available',
            'random_available',
            'best_overall_position',
            'best_position'
        ]

        self.selection_scores = {}
        self.selection_observed_scores = {}
        self.selection_counts = {}

        self.removal_scores = {}
        self.removal_observed_scores = {}
        self.removal_counts = {}

        self.insertion_scores = {}
        self.insertion_observed_scores = {}
        self.insertion_counts = {}

        # Keep track of last removed nodes for LRFI strategy
        self.last_removed_nodes = deque(maxlen=self.params.get('lrfi_count', 10))



    def _compute_route_time(self, route):
        dm = self.problem.distance_matrix
        time = 0.0
        for i in range(len(route)-1):
            time += dm[route[i]][route[i+1]]
        return time

    def _compute_solution_travel_time(self, solution):
        return sum(self._compute_route_time(r) for r in solution)

    def _is_feasible_insertion(self, route, pos, node):
        # Check feasibility of inserting node at route[pos]
        # For simplicity, assume feasibility = route time <= Tmax after insertion
        dm = self.problem.distance_matrix
        new_route = route[:pos] + [node] + route[pos:]
        return self._compute_route_time(new_route) <= self.problem.Tmax

    def _insert_node_in_position(self, route, pos, node):
        route.insert(pos, node)

    def _incremental_time_for_insertion(self, route, pos, node):
        dm = self.problem.distance_matrix
        before = dm[route[pos-1]][route[pos]]
        after = dm[route[pos-1]][node] + dm[node][route[pos]]
        return after - before
    

    def detect_subtours(self, x_sol, V, A, start_node, end_node):
        """
        Detect subtours in the current solution using Tarjan's algorithm.
        
        :param x_sol: Dictionary with keys as (i,j) tuples and values as x[i,j].
        :param V: Set of all nodes.
        :param A: Set of all arcs.
        :param start_node: Start depot node ID.
        :param end_node: End depot node ID.
        :return: List of sets, each representing a subtour.
        """
        graph = defaultdict(list)
        for (i, j), val in x_sol.items():
            if val > 0.9:
                graph[i].append(j)

        # Tarjan's algorithm for SCCs
        index = {}
        lowlink = {}
        stack = []
        onstack = set()
        sccs = []
        current_index = 0

        def strongconnect(v):
            nonlocal current_index
            index[v] = current_index
            lowlink[v] = current_index
            current_index += 1
            stack.append(v)
            onstack.add(v)

            for w in graph[v]:
                if w not in index:
                    strongconnect(w)
                    lowlink[v] = min(lowlink[v], lowlink[w])
                elif w in onstack:
                    lowlink[v] = min(lowlink[v], index[w])

            # If v is a root node, pop the stack and generate an SCC
            if lowlink[v] == index[v]:
                scc = set()
                while True:
                    w = stack.pop()
                    onstack.remove(w)
                    scc.add(w)
                    if w == v:
                        break
                # **Remove the exclusion of start_node and end_node**
                if len(scc) > 1:
                    sccs.append(scc)

        for v in V:
            if v not in index:
                strongconnect(v)

        return sccs


    def srop_subtour_callback(self, model, where):
        """
        Callback function to eliminate subtours in the SROP model.
        
        :param model: The Gurobi model.
        :param where: The callback context.
        """
        if where == GRB.Callback.MIPSOL:
            x = model._x
            V = model._V
            A = model._A
            o = model._start_node
            d = model._end_node

            # Retrieve the current solution
            x_sol = {}
            for (i, j) in A:
                val = model.cbGetSolution(x[i, j])
                x_sol[(i, j)] = val

            # Detect subtours
            subtours = self.detect_subtours(x_sol, V, A, o, d)

            # Add lazy constraints to eliminate each subtour found
            for S in subtours:
                lhs = gp.LinExpr()
                for i in S:
                    for j in S:
                        if (i, j) in A and i != j:
                            lhs.add(x[i, j], 1)
                model.cbLazy(lhs <= len(S) - 1)
                logging.info(f"Added subtour elimination constraint for subset: {S}")




    def node_elimination_procedure(self):
        """
        Apply a node elimination procedure.
        This procedure removes nodes that cannot be serviced within the allowed time budget.
        A node i is eliminated if even the direct route s -> i -> e exceeds Tmax.
        """
        logging.info("Applying node elimination procedure...")

        # Extract relevant information from the problem instance
        N = self.problem.N
        distance_matrix = self.problem.distance_matrix
        Tmax = self.problem.Tmax

        start_node = 0
        end_node = N - 1

        # Mandatory nodes (start and end) cannot be eliminated
        # We'll create a new list of nodes that remain feasible after elimination
        feasible_nodes = []

        # For each node, check feasibility:
        for node in self.problem.nodes:
            i = node['id']
            
            # Skip start and end nodes as they must remain
            if i == start_node or i == end_node:
                feasible_nodes.append(node)
                continue

            # Compute minimal route time s -> i -> e
            T_sie = distance_matrix[start_node][i] + distance_matrix[i][end_node]

            if T_sie <= Tmax:
                # This node could potentially be serviced in some feasible route
                feasible_nodes.append(node)
            else:
                # Node cannot be serviced within Tmax, so it is eliminated
                logging.info(f"Eliminating node {i} due to infeasibility (s->i->e exceeds Tmax).")

        # Update the problem instance with the reduced set of nodes
        self.problem.nodes = feasible_nodes
        # Update N to reflect the new number of nodes
        self.problem.N = len(feasible_nodes)

        # Create a dictionary keyed by node id for direct access:
        self.problem.node_dict = {node['id']: node for node in self.problem.nodes}

        logging.info("Node elimination procedure completed.")
    
    def construct_initial_solution(self):
        logging.info("Constructing initial solution using the nearest neighbor algorithm...")

        start_node = self.problem.original_start_id
        end_node = self.problem.original_end_id

        unvisited = {nid for nid in self.problem.node_dict.keys() if nid not in {start_node, end_node}}
        paths = [[start_node, end_node] for _ in range(self.problem.P)]
        current_times = []
        for p in range(self.problem.P):
            initial_time = self.problem.distance_matrix[start_node][end_node]
            current_times.append(initial_time)

        current_positions = [start_node for _ in range(self.problem.P)]

        while unvisited:
            improvement = False
            for p in range(self.problem.P):
                last_node = current_positions[p]
                best_node = None
                best_score = -1
                best_distance = float('inf')

                for node_id in unvisited:
                    distance = self.problem.distance_matrix[last_node][node_id]
                    distance_to_end = self.problem.distance_matrix[node_id][end_node]
                    new_total_time = current_times[p] - self.problem.distance_matrix[last_node][end_node] + distance + distance_to_end

                    if new_total_time <= self.problem.Tmax:
                        node_score = self.problem.node_dict[node_id]['score']  
                        heuristic = node_score / (distance + 1)
                        if heuristic > best_score:
                            best_score = heuristic
                            best_node = node_id
                            best_distance = distance

                if best_node is not None:
                    paths[p].insert(-1, best_node)
                    unvisited.remove(best_node)
                    current_times[p] = current_times[p] - self.problem.distance_matrix[last_node][end_node] + best_distance + self.problem.distance_matrix[best_node][end_node]
                    current_positions[p] = best_node
                    improvement = True
                    logging.info(f"Path {p + 1}: Added node {best_node} (Score: {self.problem.node_dict[best_node]['score']}, Distance: {best_distance:.2f})")

            if not improvement:
                logging.info("No further nodes can be added without exceeding Tmax.")
                break

        for p in range(self.problem.P):
            if paths[p][-1] != end_node:
                paths[p].append(end_node)
                logging.info(f"Path {p + 1}: Ensured ending at node {end_node}. Total time: {current_times[p]:.2f}")
                self.add_route_if_unique(p)

        self.s_best = paths
        self.s_adm = copy.deepcopy(paths)
        logging.info("Initial solution constructed with time budget consideration.")
        return paths
        
    def initialize_scores(self):
        """
        Initialize the scores of the node selection strategy and the removal and insertion operators.
        """
        logging.info("Initializing scores for strategies and operators...")

        # Each strategy/operator starts with a score of 1.0 as per the described approach
        for strat in self.selection_strategies:
            self.selection_scores[strat] = 1.0
            self.selection_observed_scores[strat] = 0.0
            self.selection_counts[strat] = 0

        for rem in self.removal_ops:
            self.removal_scores[rem] = 1.0
            self.removal_observed_scores[rem] = 0.0
            self.removal_counts[rem] = 0

        for ins in self.insertion_ops:
            self.insertion_scores[ins] = 1.0
            self.insertion_observed_scores[ins] = 0.0
            self.insertion_counts[ins] = 0
    
    def select_node_selection_strategy(self):
        """
        Select a node selection strategy based on current scores.
        A roulette wheel selection is performed:
        P_k = p_k / sum_of_all_p
        The strategies are:
        1. dynamic_profit_per_time
        2. highest_profit
        3. random
        4. lrfi
        """
        logging.info("Selecting node selection strategy...")
        total_score = sum(self.selection_scores.values())
        r = random.uniform(0, total_score)
        cumulative = 0.0
        for strat, score in self.selection_scores.items():
            cumulative += score
            if r <= cumulative:
                return strat
        # Fallback in case of numerical issues
        return self.selection_strategies[0]
    
    def select_removal_insertion_operators(self):
        """
        Select removal and insertion operators based on current scores via roulette wheel selection.
        """
        logging.info("Selecting removal and insertion operators...")

        # Removal operator selection
        total_score_rem = sum(self.removal_scores.values())
        r_rem = random.uniform(0, total_score_rem)
        cumulative_rem = 0.0
        selected_removal = None
        for rem, score in self.removal_scores.items():
            cumulative_rem += score
            if r_rem <= cumulative_rem:
                selected_removal = rem
                break

        # Insertion operator selection
        total_score_ins = sum(self.insertion_scores.values())
        r_ins = random.uniform(0, total_score_ins)
        cumulative_ins = 0.0
        selected_insertion = None
        for ins, score in self.insertion_scores.items():
            cumulative_ins += score
            if r_ins <= cumulative_ins:
                selected_insertion = ins
                break

        return selected_removal, selected_insertion
    
    def remove_nodes(self, solution, b, removal_operator,ratio_sequence_to_remove):
            logging.info(f"Removing nodes using {removal_operator}...")
            
            if removal_operator == 'random_removal':
                removed = self._remove_random_nodes(solution, b)
            elif removal_operator == 'lowest_profit_removal':
                removed = self._remove_lowest_profit_nodes(solution, b)
            elif removal_operator == 'largest_saving_removal':
                removed = self._remove_largest_saving_nodes(solution, b)
            elif removal_operator == 'route_removal':
                removed = self._remove_route(solution)
            elif removal_operator == 'sequence_removal':
                removed = self._remove_sequence(solution, ratio_sequence_to_remove)
            else:
                # Default to random removal if unknown
                removed = self._remove_random_nodes(solution, b)
            
            # Update solution by removing chosen nodes
            for route in solution:
                route[:] = [x for x in route if x not in removed]

            # Update LRFI queue
            for n in removed:
                self.last_removed_nodes.appendleft(n)
            
            return solution

    def _remove_random_nodes(self, solution, b):
        nodes = [n for route in solution for n in route if n not in (self.problem.original_start_id, self.problem.original_end_id)]
        removed = set()
        for _ in range(min(b, len(nodes))):
            n = random.choice(nodes)
            nodes.remove(n)
            removed.add(n)
        return removed

    def _remove_lowest_profit_nodes(self, solution, b):
        # Weighted removal based on lowest profit: nodes with lower profit are more likely to be chosen
        nodes = [n for route in solution for n in route if n not in (self.problem.original_start_id, self.problem.original_end_id)]
        if not nodes:
            return set()
        
        # Sort by profit ascending
        nodes.sort(key=lambda x: self.problem.node_dict[x]['score'])
        total_profit = sum(self.problem.node_dict[n]['score'] for n in nodes) + 1e-9
        removed = set()
        for _ in range(min(b, len(nodes))):
            # Roulette wheel giving higher chance to lower profit nodes
            r = random.uniform(0, total_profit)
            cumulative = 0.0
            chosen = nodes[0]
            for n in nodes:
                cumulative += self.problem.node_dict[n]['score']
                if r <= cumulative:
                    chosen = n
                    break
            removed.add(chosen)
            nodes.remove(chosen)
            total_profit -= self.problem.node_dict[chosen]['score']
        return removed

    def _remove_largest_saving_nodes(self, solution, b):
        # Compute saving in travel time for removing each node: D(s) - D(s\i)
        # First compute D(s)
        original_time = self._compute_solution_travel_time(solution)

        # For each node, compute saving
        candidates = []
        for r_idx, route in enumerate(solution):
            for i in range(1, len(route)-1):
                n = route[i]
                # Temporarily remove node n and compute new time
                temp_route = route[:i] + route[i+1:]
                temp_sol = solution[:r_idx] + [temp_route] + solution[r_idx+1:]
                new_time = self._compute_solution_travel_time(temp_sol)
                saving = original_time - new_time
                candidates.append((n, saving))
        
        if not candidates:
            return set()

        # Sort by saving descending, then use roulette wheel with weights = saving
        total_saving = sum(s for _, s in candidates) + 1e-9
        removed = set()
        nodes_list = candidates[:]
        # Remove b nodes with preference to largest saving
        for _ in range(min(b, len(nodes_list))):
            r = random.uniform(0, total_saving)
            cumulative = 0.0
            chosen = nodes_list[0][0]
            for (nd, sv) in nodes_list:
                cumulative += sv
                if r <= cumulative:
                    chosen = nd
                    break
            removed.add(chosen)
            # Remove chosen from candidate list and adjust total_saving
            for idx, (nd, sv) in enumerate(nodes_list):
                if nd == chosen:
                    total_saving -= sv
                    del nodes_list[idx]
                    break
        return removed

    def _remove_route(self, solution):
        # Randomly select a route (other than maybe a trivial one) and remove all nodes from it
        non_empty_routes = [r for r in solution if len(r) > 2]
        if not non_empty_routes:
            # If no non-empty routes, nothing to remove
            return set()
        route = random.choice(non_empty_routes)
        # Remove all non-depot nodes
        removed = set(route[1:-1])
        return removed

    def _remove_sequence(self, solution, ratio_sequence_to_remove):
        route = random.choice(solution)
        start_idx = random.randint(2, 2 + int(ratio_sequence_to_remove*len(route)) )
        removed = set(route[start_idx:start_idx+ int(ratio_sequence_to_remove*len(route))])
        return removed

    # insertion operators
    def insert_nodes(self, solution, insertion_operator, strategy):
        logging.info(f"Inserting nodes using {insertion_operator} with strategy {strategy}...")
        
        # Determine nodes to insert based on strategy (already chosen and ordered)
        visited = {nid for r in solution for nid in r}
        start_node = self.problem.original_start_id
        end_node =  self.problem.original_end_id
        unvisited = set(self.problem.node_dict.keys()) - visited - {start_node, end_node}

        # Strategy decides the order in which nodes are considered. This is handled separately.
        # Here we assume the `strategy` has already selected insertion_order in a previous step.
        # For demonstration, let's assume we have the insertion_order from node selection strategy step:
        # In real code, you'd integrate your node selection strategy logic here.
        insertion_order = self._get_insertion_order(unvisited, strategy)

        # Apply the chosen insertion operator to insert nodes in the given order
        if insertion_operator == 'first_available':
            solution = self._insert_nodes_first_available(solution, insertion_order)
        elif insertion_operator == 'last_available':
            solution = self._insert_nodes_last_available(solution, insertion_order)
        elif insertion_operator == 'random_available':
            solution = self._insert_nodes_random_available(solution, insertion_order)
        elif insertion_operator == 'best_overall_position':
            solution = self._insert_nodes_best_overall_position(solution, insertion_order)
        elif insertion_operator == 'best_position':
            solution = self._insert_nodes_best_position(solution, insertion_order)
        else:
            # Default to a simple operator, e.g., first_available
            solution = self._insert_nodes_first_available(solution, insertion_order)
        
        return solution

    def _get_insertion_order(self, unvisited, strategy):
        """
        Determine the order in which nodes from 'unvisited' should be considered for insertion
        based on the selected node selection strategy.
        Strategies:
        - dynamic_profit_per_time
        - highest_profit
        - random
        - lrfi
        """
        if not unvisited:
            return []

        if strategy == 'dynamic_profit_per_time':
            # Compute p_i / D(s+i) for each node i
            candidates = []
            for n in unvisited:
                profit = self.problem.node_dict[n]['score']
                incremental_time = self._compute_min_incremental_time(self.s_adm, n)
                ratio = profit / incremental_time if incremental_time > 0 else float('inf')
                candidates.append((n, ratio))
            # Sort by ratio descending
            candidates.sort(key=lambda x: x[1], reverse=True)
            insertion_list = [n for n, _ in candidates]

        elif strategy == 'highest_profit':
            # Roulette wheel based on profit
            temp_nodes = list(unvisited)
            insertion_list = []
            total_profit = sum(self.problem.node_dict[n]['score'] for n in temp_nodes) + 1e-9
            while temp_nodes:
                r = random.uniform(0, total_profit)
                cumulative = 0.0
                chosen = temp_nodes[0]
                for node_id in temp_nodes:
                    cumulative += self.problem.node_dict[node_id]['score']
                    if r <= cumulative:
                        chosen = node_id
                        break
                insertion_list.append(chosen)
                temp_nodes.remove(chosen)
                total_profit -= self.problem.node_dict[chosen]['score']

        elif strategy == 'random':
            insertion_list = list(unvisited)
            random.shuffle(insertion_list)

        elif strategy == 'lrfi':
            # Insert last removed nodes first if they are still unvisited
            lrfi_candidates = [n for n in self.last_removed_nodes if n in unvisited]
            # Remove them from unvisited
            remainder = list(unvisited - set(lrfi_candidates))
            # For remainder, fallback to random (or another strategy if desired)
            random.shuffle(remainder)
            insertion_list = lrfi_candidates + remainder

        else:
            # Default fallback - random
            insertion_list = list(unvisited)
            random.shuffle(insertion_list)

        return insertion_list


    def _compute_min_incremental_time(self, solution, node):
        """
        Compute the minimal incremental travel time of inserting 'node' anywhere in the given 'solution'.
        """
        min_increment = float('inf')
        dm = self.problem.distance_matrix
        for r_idx, route in enumerate(solution):
            for i in range(len(route)-1):
                a, b = route[i], route[i+1]
                current_time = dm[a][b]
                new_time = dm[a][node] + dm[node][b]
                increment = new_time - current_time
                if increment < min_increment:
                    min_increment = increment
        return min_increment if min_increment != float('inf') else 1e9  # Large number if no feasible insertion

    # Insertion methods
    def _insert_nodes_first_available(self, solution, nodes):
        # For each node, insert at the first feasible position in some route
        for node in nodes:
            # Try each route in order, from start to end
            inserted = False
            for route in solution:
                for pos in range(1, len(route)):
                    if self._is_feasible_insertion(route, pos, node):
                        self._insert_node_in_position(route, pos, node)
                        inserted = True
                        break
                if inserted:
                    break
        return solution

    def _insert_nodes_last_available(self, solution, nodes):
        # For each node, insert at the last feasible position scanning routes from end to start
        for node in nodes:
            inserted = False
            for route in solution:
                for pos in range(len(route)-1, 0, -1):
                    if self._is_feasible_insertion(route, pos, node):
                        self._insert_node_in_position(route, pos, node)
                        inserted = True
                        break
                if inserted:
                    break
        return solution

    def _insert_nodes_random_available(self, solution, nodes):
        # For each node, find all feasible positions and choose one at random
        for node in nodes:
            candidates = []
            for r_idx, route in enumerate(solution):
                for pos in range(1, len(route)):
                    if self._is_feasible_insertion(route, pos, node):
                        candidates.append((r_idx, pos))
            if candidates:
                r_idx, pos = random.choice(candidates)
                self._insert_node_in_position(solution[r_idx], pos, node)
        return solution

    def _insert_nodes_best_overall_position(self, solution, nodes):
        # For each node, test all feasible insertions and choose the one minimizing total solution travel time
        # This can be expensive, but we do it as described.
        original_time = self._compute_solution_travel_time(solution)
        for node in nodes:
            best_delta = float('inf')
            best_route = None
            best_pos = None
            for r_idx, route in enumerate(solution):
                for pos in range(1, len(route)):
                    if self._is_feasible_insertion(route, pos, node):
                        # Insert temporarily and check total time
                        temp_sol = copy.deepcopy(solution)
                        self._insert_node_in_position(temp_sol[r_idx], pos, node)
                        new_time = self._compute_solution_travel_time(temp_sol)
                        delta = new_time - original_time
                        if delta < best_delta:
                            best_delta = delta
                            best_route = r_idx
                            best_pos = pos
            if best_route is not None:
                self._insert_node_in_position(solution[best_route], best_pos, node)
                # Update original_time after insertion
                original_time += best_delta
        return solution

    def _insert_nodes_best_position(self, solution, nodes):
        # For each node, find the insertion that minimizes local incremental time
        for node in nodes:
            best_increment = float('inf')
            best_route = None
            best_pos = None
            for r_idx, route in enumerate(solution):
                for pos in range(1, len(route)):
                    if self._is_feasible_insertion(route, pos, node):
                        increment = self._incremental_time_for_insertion(route, pos, node)
                        if increment < best_increment:
                            best_increment = increment
                            best_route = r_idx
                            best_pos = pos
            if best_route is not None:
                self._insert_node_in_position(solution[best_route], best_pos, node)
        return solution
    def evaluate_solution(self, solution):
        visited = set()
        for route in solution:
            for nid in route:
                visited.add(nid)
                
        total_score = 0.0
        for nid in visited:
            total_score += self.problem.node_dict[nid]['score']
        
        return total_score
    
    def apply_local_search(self, solution):
        """
        Apply local search procedures on the solution, as described in the paper:
        1. Apply a complete 2-opt procedure on each route.
        2. Try inserting randomly selected non-inserted nodes using "Best position insertion".
        3. Randomly select two inserted nodes i, j and one non-inserted node k, remove i and j,
        then try inserting k, i, j in a way to improve the solution.
        4. Randomly select two inserted nodes from different routes and try to swap them.
        
        If the solution is not improved after these moves, it gets rejected and the original solution is returned.
        """
        logging.info("Applying local search procedures...")
        
        original_solution = copy.deepcopy(solution)
        original_value = self.evaluate_solution(original_solution)
        best_solution = copy.deepcopy(original_solution)
        best_value = original_value

        # 1. Apply complete 2-opt to each route
        sol_2opt = self._apply_2opt(best_solution)
        val_2opt = self.evaluate_solution(sol_2opt)
        if val_2opt > best_value:
            best_solution = copy.deepcopy(sol_2opt)
            best_value = val_2opt
        
        # 2. Try inserting non-inserted nodes with "Best position insertion"
        sol_insert = self._try_inserting_non_inserted(best_solution)
        val_insert = self.evaluate_solution(sol_insert)
        if val_insert > best_value:
            best_solution = copy.deepcopy(sol_insert)
            best_value = val_insert
        
        # 3. Remove two inserted nodes i, j and insert a non-inserted node k, then i and j again
        sol_replace = self._replace_two_inserted_with_one_non_inserted(best_solution)
        val_replace = self.evaluate_solution(sol_replace)
        if val_replace > best_value:
            best_solution = copy.deepcopy(sol_replace)
            best_value = val_replace
        
        # 4. Swap two inserted nodes from different routes
        sol_swap = self._swap_inserted_nodes(best_solution)
        val_swap = self.evaluate_solution(sol_swap)
        if val_swap > best_value:
            best_solution = copy.deepcopy(sol_swap)
            best_value = val_swap

        # Check if any improvement was made
        if best_value > original_value:
            logging.info("Local search improved the solution.")
            return best_solution
        else:
            logging.info("Local search did not improve the solution. Reverting to original.")
            return original_solution

    def _apply_2opt(self, solution):

        """
        Apply a complete 2-opt procedure to each route in the solution to reduce travel time.
        The 2-opt heuristic repeatedly attempts to find two edges (i,i+1) and (j,j+1) in a route,
        and if reversing the segment between them reduces the total route travel time, it applies that change.
        
        :param solution: A list of routes (each route is a list of node IDs).
        :return: A new solution after attempting 2-opt improvements on each route.
        """
        improved_solution = [list(route) for route in solution]  # Deep copy
        improved = True
        while improved:
            improved = False
            for r_idx, route in enumerate(improved_solution):
                # If the route is too short, 2-opt won't apply
                if len(route) < 4:
                    continue

                best_improvement = 0
                best_i = None
                best_j = None
                current_time = self._compute_route_time(route)

                # Try all pairs (i, j) with i < j-1 to ensure non-overlapping edges
                for i in range(1, len(route)-2):
                    for j in range(i+2, len(route)-1):
                        # Current edges: (route[i-1], route[i]) and (route[j], route[j+1])
                        # After reversal: (route[i-1], route[j]) and (route[i], route[j+1])
                        old_dist = (self.problem.distance_matrix[route[i-1]][route[i]] +
                                    self.problem.distance_matrix[route[j]][route[j+1]])
                        new_dist = (self.problem.distance_matrix[route[i-1]][route[j]] +
                                    self.problem.distance_matrix[route[i]][route[j+1]])
                        
                        improvement = old_dist - new_dist
                        if improvement > best_improvement:
                            best_improvement = improvement
                            best_i = i
                            best_j = j

                if best_improvement > 0 and best_i is not None and best_j is not None:
                    # Apply the 2-opt reversal
                    route[best_i:best_j+1] = reversed(route[best_i:best_j+1])
                    improved = True

        return improved_solution

    def _try_inserting_non_inserted(self, solution):
        """
        Randomly select some non-inserted nodes and try inserting them
        with the "Best position insertion" operator.

        Best position insertion:
        - For each selected non-inserted node, try all feasible insertions (route, pos).
        - Insert it where it yields the highest improvement in the solution's objective value.
        
        :param solution: Current solution (list of routes)
        :return: Improved solution after attempting these insertions
        """
        improved_solution = copy.deepcopy(solution)
        original_value = self.evaluate_solution(improved_solution)
        best_solution = copy.deepcopy(improved_solution)
        best_value = original_value

        visited = {n for route in best_solution for n in route}
        start_node = self.problem.original_start_id
        end_node = self.problem.original_end_id
        all_nodes = set(self.problem.node_dict.keys())
        non_inserted = list(all_nodes - visited - {start_node, end_node})

        # Shuffle non_inserted for randomness and limit the number of attempts
        random.shuffle(non_inserted)
        # For efficiency, we only attempt a subset, say up to 10 nodes
        non_inserted = non_inserted[:10]

        for node in non_inserted:
            # Attempt to insert this node in the best possible position
            temp_best_solution = None
            temp_best_value = best_value

            # Try all routes and all feasible positions
            for r_idx, route in enumerate(best_solution):
                for pos in range(1, len(route)):  # Avoid placing before start or after end
                    if self._is_feasible_insertion(route, pos, node):
                        # Test insertion
                        trial_solution = copy.deepcopy(best_solution)
                        self._insert_node_in_position(trial_solution[r_idx], pos, node)
                        val = self.evaluate_solution(trial_solution)
                        
                        if val > temp_best_value:
                            temp_best_value = val
                            temp_best_solution = trial_solution

            # After checking all positions, if we found an improvement, update best_solution
            if temp_best_solution is not None and temp_best_value > best_value:
                best_value = temp_best_value
                best_solution = temp_best_solution

        return best_solution


    def _replace_two_inserted_with_one_non_inserted(self, solution):
        """
        Randomly select two inserted nodes i and j and one non-inserted node k.
        Remove i and j, then try inserting k, followed by reinserting i and j
        to improve the solution value.
        
        :param solution: Current solution
        :return: Modified solution after this heuristic
        """
        original_solution = copy.deepcopy(solution)
        original_value = self.evaluate_solution(original_solution)
        best_solution = copy.deepcopy(original_solution)
        best_value = original_value

        visited = {n for route in original_solution for n in route}
        start_node = self.problem.original_start_id
        end_node = self.problem.original_end_id
        all_nodes = set(self.problem.node_dict.keys())
        inserted = list(visited - {start_node, end_node})
        non_inserted = list(all_nodes - visited - {start_node, end_node})

        # Check if we have enough nodes to attempt
        if len(inserted) < 2 or len(non_inserted) < 1:
            return best_solution  # Cannot perform this operation

        # Select i, j from inserted, k from non_inserted
        i_node, j_node = random.sample(inserted, 2)
        k_node = random.choice(non_inserted)

        # Remove i_node and j_node
        temp_solution = copy.deepcopy(original_solution)
        for route in temp_solution:
            route[:] = [x for x in route if x not in (i_node, j_node)]

        # Insert k_node (best position insertion)
        inserted_k = False
        temp_best_value = self.evaluate_solution(temp_solution)
        temp_best_sol = copy.deepcopy(temp_solution)

        for r_idx, route in enumerate(temp_solution):
            for pos in range(1, len(route)):
                if self._is_feasible_insertion(route, pos, k_node):
                    trial_sol = copy.deepcopy(temp_solution)
                    self._insert_node_in_position(trial_sol[r_idx], pos, k_node)
                    val = self.evaluate_solution(trial_sol)
                    if val > temp_best_value:
                        temp_best_value = val
                        temp_best_sol = trial_sol
                        inserted_k = True

        if not inserted_k:
            # Could not insert k anywhere, revert
            return best_solution

        # Now temp_best_sol is the solution with k inserted at its best position
        # Try reinserting i and j using best position insertion
        def best_position_insert(node, current_sol, current_val):
            best_val = current_val
            best_ins_sol = copy.deepcopy(current_sol)
            for r_idx, route in enumerate(current_sol):
                for pos in range(1, len(route)):
                    if self._is_feasible_insertion(route, pos, node):
                        trial_sol = copy.deepcopy(current_sol)
                        self._insert_node_in_position(trial_sol[r_idx], pos, node)
                        val = self.evaluate_solution(trial_sol)
                        if val > best_val:
                            best_val = val
                            best_ins_sol = trial_sol
            return best_ins_sol, best_val

        # Insert i_node
        temp_sol_i, val_i = best_position_insert(i_node, temp_best_sol, temp_best_value)
        # Insert j_node (in the solution after i_node is reinserted)
        temp_sol_j, val_j = best_position_insert(j_node, temp_sol_i, val_i)

        # Check improvement
        if val_j > best_value:
            best_value = val_j
            best_solution = temp_sol_j

        return best_solution

    def _swap_inserted_nodes(self, solution):
        """
        Randomly select two inserted nodes served by different routes and attempt to swap them
        to reduce total travel time.
        
        :param solution: Current solution (list of routes)
        :return: Modified solution after attempting swaps
        """
        original_solution = copy.deepcopy(solution)
        original_time = self._compute_solution_travel_time(original_solution)
        best_solution = copy.deepcopy(original_solution)
        best_time = original_time

        start_node = self.problem.original_start_id
        end_node = self.problem.original_end_id

        # Identify inserted nodes and their positions
        inserted_positions = []  # list of (node, route_index, position_in_route)
        for r_idx, route in enumerate(original_solution):
            for pos, nid in enumerate(route):
                if nid not in (start_node, end_node):
                    inserted_positions.append((nid, r_idx, pos))
        
        if len(inserted_positions) < 2:
            # Not enough inserted nodes to attempt a swap
            return best_solution

        # Limit attempts for efficiency
        attempts = min(10, (len(inserted_positions)*(len(inserted_positions)-1))//2)
        tried_pairs = set()

        for _ in range(attempts):
            n1, r1, p1 = random.choice(inserted_positions)
            n2, r2, p2 = random.choice(inserted_positions)
            
            # Ensure distinct nodes and different routes
            if n1 == n2 or r1 == r2:
                continue
            
            pair_key = tuple(sorted([n1, n2]))
            if pair_key in tried_pairs:
                continue
            tried_pairs.add(pair_key)

            # Attempt swap
            trial_solution = copy.deepcopy(best_solution)
            trial_solution[r1][p1], trial_solution[r2][p2] = trial_solution[r2][p2], trial_solution[r1][p1]

            val = self._compute_solution_travel_time(trial_solution)
            # Since we want to reduce travel time, an improvement means val < best_time
            if val < best_time:
                best_time = val
                best_solution = trial_solution

        return best_solution

    

    def generate_and_solve_SROP(self, solution):
        """
        Generate and solve a  Sub-Route Optimization Problem (SROP) with potential subtour elimination callback.
        
        :param solution: Current solution (a list of routes)
        :return: Optimized solution
        """
        logging.info("Generating and solving SORP...")

        # Select a route to optimize (e.g., longest route)
        if not solution or all(len(r) <= 2 for r in solution):
            return copy.deepcopy(solution)
        for route_to_optimize in solution: #= max(solution, key=lambda r: len(r))
            start_node = route_to_optimize[0]
            end_node = route_to_optimize[-1]

            # Identify all nodes currently served by the solution
            served_nodes = set()
            for route in solution:
                served_nodes.update(route)
            served_nodes.discard(start_node)
            served_nodes.discard(end_node)

            # Identify unselected (unserved) nodes
            all_nodes = set(self.problem.node_dict.keys())
            unselected_nodes = all_nodes - served_nodes - {start_node, end_node}

            # Define V as the union of nodes in the selected route and unselected nodes
            V = set(route_to_optimize).union(unselected_nodes)


            # Build arcs A and data tij, pi
            A = []
            tij = {}
            pi = {}
            node_list = list(V)

            for i in node_list:
                pi[i] = self.problem.node_dict[i]['score'] if i not in (start_node, end_node) else 0.0
                for j in node_list:
                    if i != j:
                        A.append((i,j))
                        tij[i,j] = self.problem.distance_matrix[i][j]

            # Compute original route time to set Dmax
            original_time = 0.0
            for i in range(len(route_to_optimize)-1):
                original_time += self.problem.distance_matrix[route_to_optimize[i]][route_to_optimize[i+1]]
            Dmax = original_time

            # Build SROP model
            m = gp.Model("SROP")
            m.Params.LazyConstraints = 1

            x = m.addVars(A, vtype=GRB.BINARY, name="x")
            y = m.addVars(V, vtype=GRB.BINARY, name="y")

            # Objective: maximize profit
            m.setObjective(gp.quicksum(pi[i]*y[i] for i in V if i not in {start_node, end_node}), GRB.MAXIMIZE)

            # Start depot:
            m.addConstr(gp.quicksum(x[start_node,j] for j in V if (start_node,j) in A) == 1)
            # End depot:
            m.addConstr(gp.quicksum(x[i,end_node] for i in V if (i,end_node) in A) == 1)

            # Flow and linking:
            for i in V:
                if i not in {start_node, end_node}:
                    m.addConstr(gp.quicksum(x[i,j] for j in V if (i,j) in A) == y[i])
                    m.addConstr(gp.quicksum(x[j,i] for j in V if (j,i) in A) == y[i])

            # Time constraint:
            m.addConstr(gp.quicksum(tij[i,j]*x[i,j] for (i,j) in A) <= Dmax)

            # Store data for callback
            m._x = x
            m._V = V
            m._A = A
            m._start_node = start_node
            m._end_node = end_node

            m.optimize(self.srop_subtour_callback)

            if m.status == GRB.OPTIMAL:
                logging.info("SROP Optimal solution found with objective:", m.objVal)
                x_sol = m.getAttr('x', x)
            
                # Reconstruct path
                # Step 1: Build a mapping from each node to its next node
                next_node = {}
                for (i, j), val in x_sol.items():
                    if val > 0.9:
                        if i in next_node:
                            logging.info(f"Warning: Multiple outgoing arcs detected from node {i}.")
                        next_node[i] = j
                # Step 2: Reconstruct the path using the mapping
                path = [start_node]
                current = start_node
                
                while current != end_node :
                    if current not in next_node:
                        logging.info(f"Error: No outgoing arc from node {current}. Incomplete path.")
                        break  
                    next_j = next_node[current]
                    path.append(next_j)
                    current = next_j

                
                if current != end_node:
                    logging.info(f"Error: Route reconstruction incomplete. Reached node {current} instead of {end_node}.")
                    
                
                # Step 3: Calculate the profit of the optimized route
                optimized_profit = sum(self.problem.node_dict[n]['score'] for n in path if n not in {start_node, end_node})
                original_profit = sum(self.problem.node_dict[n]['score'] for n in route_to_optimize if n not in {start_node, end_node})

                logging.info(f"Original Route Profit: {original_profit}, Optimized Route Profit: {optimized_profit}")

                # Step 4: Replace the route only if the optimized profit is greater or equal
                if optimized_profit >= original_profit:
                    new_solution = copy.deepcopy(solution)
                    route_index = solution.index(route_to_optimize)
                    new_solution[route_index] = path
                    logging.info("SROP completed and route optimized.")
                    self.add_route_if_unique(path)
                    
                else:
                    logging.info("SROP did not find an improved or equal solution. Retaining the original route.")
                    new_solution =  copy.deepcopy(solution)
            else:
                logging.info("SROP no improvement or no feasible solution found.")
                new_solution =  copy.deepcopy(solution)
            
        return new_solution

    def generate_and_solve_SPP(self):
        """
        Solve the Set Packing Problem (SPP) to select |L| routes from the collected routes that maximize total profit.

        :return: Final optimized solution (list of routes) or None if no solution is found.
        """
        logging.info("Generating and solving SPP...")

        if not self.routes_list:
            logging.warning("No routes collected for SPP.")
            return None

        R = range(len(self.routes_list))  # Route indices
        num_vehicles = self.problem.P        # Number of vehicles (paths), from ProblemInstance
        nodes = set()
        for _, _, coverage in self.routes_list:
            nodes.update(coverage.keys())

        m = gp.Model("SPP")
        m.setParam('OutputFlag', 0)  # Suppress Gurobi output for cleaner logs

        # Decision Variables
        z = m.addVars(R, vtype=GRB.BINARY, name="z")

        # Objective: Maximize total profit
        m.setObjective(gp.quicksum(self.routes_list[r][1] * z[r] for r in R), GRB.MAXIMIZE)

        # Constraints:
        # Each node is served at most once
        for i in nodes:
            m.addConstr(
                gp.quicksum(self.routes_list[r][2].get(i, 0) * z[r] for r in R) <= 1,
                name=f"node_{i}_coverage"
            )

        # Exactly |L| routes are selected
        m.addConstr(
            gp.quicksum(z[r] for r in R) == num_vehicles,
            name="exact_routes_selected"
        )

        # Optimize the model
        m.optimize()

        if m.status == GRB.OPTIMAL:
            logging.info(f"SPP Optimal solution found with total profit: {m.objVal}")
            chosen_routes = [r for r in R if z[r].X > 0.9]
            final_solution = self.build_solution_from_routes(chosen_routes)
            logging.info(f"Selected {len(final_solution)} routes for the final solution.")
            return final_solution
        else:
            logging.warning("No optimal solution found for SPP.")
            return None

        
    def add_route_if_unique(self, route):
        """
        Add a route to the routes_list and generated_routes if it's unique.
        
        :param route: List of node IDs representing the route.
        """
        # Create a frozenset of nodes to check uniqueness (order doesn't matter)
        route_nodes = frozenset(route)
        if route_nodes not in self.generated_routes:
            self.generated_routes.add(route_nodes)
            # Calculate profit (sum of node scores, excluding start and end)
            profit = sum(self.problem.node_dict[n]['score'] for n in route if n not in {self.problem.original_start_id, self.problem.original_end_id})
            # Create coverage dictionary {node:1}
            coverage = {n: 1 for n in route if n not in {self.problem.original_start_id, self.problem.original_end_id}}
            # Append to routes_list
            self.routes_list.append((route, profit, coverage))
            logging.info(f"Added new unique route with profit {profit}.")

    def build_solution_from_routes(self, chosen_route_indices):
        """
        Build the final solution from the selected routes.
        
        :param chosen_route_indices: List of indices representing chosen routes in routes_list.
        :return: Final solution as a list of routes.
        """
        final_solution = []
        for idx in chosen_route_indices:
            route = self.routes_list[idx][0]  # Extract the route
            final_solution.append(route)
        return final_solution
        
    def update_scores(self):
        """
        Update the scores of the ALNS operators and insertion strategies at the end of a segment.
        
        p_{k,q+1} = kappa * (observed_score_k / n_k) + (1 - kappa)*p_{k,q}, if n_k > 0
        If n_k = 0, score remains unchanged.
        After updating, observed scores and counts reset.
        """
        logging.info("Updating scores of the ALNS operators and insertion strategies...")

        kappa = self.params.get('kappa', 0.5)

        def update_category_scores(scores, observed_scores, counts):
            for k in scores.keys():
                n_k = counts[k]
                if n_k > 0:
                    old_score = scores[k]
                    new_score = kappa * (observed_scores[k] / n_k) + (1 - kappa) * old_score
                    scores[k] = new_score
                # Reset for next segment
                observed_scores[k] = 0.0
                counts[k] = 0

        update_category_scores(self.selection_scores, self.selection_observed_scores, self.selection_counts)
        update_category_scores(self.removal_scores, self.removal_observed_scores, self.removal_counts)
        update_category_scores(self.insertion_scores, self.insertion_observed_scores, self.insertion_counts)

        logging.info("Score update completed.")
        
    def run(self):
            # Step 1: Apply node elimination procedure
            self.node_elimination_procedure()
            
            # Step 2: Construct initial solution using the nearest neighbor algorithm
            initial_solution = self.construct_initial_solution()
            self.s_best = initial_solution
            self.s_adm = initial_solution
            
            # Initialize parameters
            self.T = self.params['T0']
            self.seg = 0
            self.iteration_best = 0
            
            # Step 4: Initialize scores
            self.initialize_scores()

            # Extract q parameters
            q1 = self.params.get('q1', 10.0)
            q2 = self.params.get('q2', 5.0)
            q3 = self.params.get('q3', 1.0)

            # Step 5: Main loop
            while self.seg < self.params['Nseg'] and self.iteration_best < self.params['iteration_best_max']:
                logging.info(f"\n--- Segment {self.seg + 1} ---")
                iteration = 0
                
                while iteration < self.params['iteration_max']:
                    logging.info(f"\nIteration {iteration + 1} within Segment {self.seg + 1}")
                    s = copy.deepcopy(self.s_adm)
                    
                    # Generate b (number of nodes to remove)
                    # For safety, ensure b does not exceed solution complexity; using a simple logic:
                    b = random.randint(1, max(1, int(0.25*sum(len(p) for p in s)-2*len(s))))
                    ratio_sequence_to_remove = self.params['ratio_sequence_to_remove']
                    logging.info(f"Number of nodes to remove: {b}")
                    
                    # Select node selection strategy
                    c = self.select_node_selection_strategy()
                    
                    # Select removal and insertion operators
                    R, I = self.select_removal_insertion_operators()
                    
                    # Count usage
                    self.selection_counts[c] += 1
                    self.removal_counts[R] += 1
                    self.insertion_counts[I] += 1

                    # Remove b nodes using R
                    s = self.remove_nodes(s, b, R, ratio_sequence_to_remove)
                    
                    # Insert nodes using I following c
                    s = self.insert_nodes(s, I, c)
                    
                    # Generate a random number d in [0,1]
                    d = random.uniform(0, 1)
                    logging.info(f"Random number d: {d}")
                    
                    # Evaluate solutions
                    f_s = self.evaluate_solution(s)
                    f_sadm = self.evaluate_solution(self.s_adm)
                    f_sbest = self.evaluate_solution(self.s_best)
                    
                    if f_s >= f_sadm or d <= math.exp((f_s - f_sadm) / self.T):
                        logging.info("Solution accepted based on acceptance criteria.")
                        
                        # Determine q increment based on improvement
                        if f_s > f_sbest:
                            q_value = q1
                        elif f_s > f_sadm:
                            q_value = q2
                        else:
                            q_value = q3

                        # Increment observed scores
                        self.selection_observed_scores[c] += q_value
                        self.removal_observed_scores[R] += q_value
                        self.insertion_observed_scores[I] += q_value

                        # If f(s) > f(sadm), apply local search
                        if f_s > f_sadm:
                            s = self.apply_local_search(s)
                            f_s = self.evaluate_solution(s)
                            logging.info("Applied local search.")

                        # If f(s) > f(s_best), update sbest and reset iteration_best
                        if f_s > f_sbest:
                            logging.info("New best solution found. Generating and solving SROP...")
                            s = self.generate_and_solve_SROP(s) 
                            self.s_best = copy.deepcopy(s)
                            self.iteration_best = 0
                            logging.info("s_best updated.")
                        else:
                            self.iteration_best += 1
                            logging.info(f"Iteration best incremented to {self.iteration_best}.")
                        
                        self.s_adm = copy.deepcopy(s)
                    else:
                        # Not accepted
                        self.iteration_best += 1
                        logging.info(f"Solution not accepted. Iteration best incremented to {self.iteration_best}.")
                    
                    # Temperature check
                    if self.T <= self.params['T_min']:
                        logging.info("Temperature below minimum. Resetting temperature and generating SPP...")
                        self.T = self.params['T0']
                        logging.info(self.routes_list)
                        s = self.generate_and_solve_SPP()
                        
                        if self.evaluate_solution(s) > self.evaluate_solution(self.s_best):
                            self.iteration_best = 0
                            logging.info("Improved solution found after SPP.")
                        
                        self.s_best = copy.deepcopy(s)
                        self.s_adm = copy.deepcopy(s)
                    
                    # Update temperature and iteration
                    self.T *= self.params['cooling_rate']
                    iteration += 1
                    logging.info(f"Temperature updated to {self.T}.")
                
                # Update scores at the end of the segment
                self.update_scores()
                
                # Increment segment
                self.seg += 1
                logging.info(f"Segment {self.seg} completed.")
            
            # Generate and solve a final SPP
            logging.info("Route List: ",  self.routes_list)
            logging.info("\nGenerating and solving final SPP...")
            self.s_best = self.generate_and_solve_SPP()
            
            if self.s_best is None:
                logging.warning("No best solution found. Returning a trivial empty solution to avoid None.")
                self.s_best = [[self.problem.original_start_id, self.problem.original_end_id] for _ in range(self.problem.P) ]
    
            # Final solution
            logging.info("Final solution obtained.")
            return self.s_best


In [12]:
if __name__ == "__main__":
    # Define the path to your TOP instance data file
    file_path = 'Set_100_234/p4.2.a.txt' # 3 paths does not properly work currently!

    parameters = {
        'T0': 95,              # Initial temperature for the simulated annealing mechanism.
        'T_min': 0.0001,              # Minimum temperature below which we reset temperature and solve a SPP.
        'cooling_rate': 0.9999,    # The factor by which the temperature is multiplied after each iteration.
        'Nseg': 100,              # Number of segments (run segments) after which score updates occur.
        'iteration_max': 1000,  # Maximum number of iterations per segment.
        'iteration_best_max': 50,# Maximum number of consecutive iterations without improvement before stopping.
        'kappa': 0.85,            # Reaction factor for updating operator/strategy scores.
        'q1': 20.0,              # Score increment if a new best solution is found.
        'q2': 5.0,               # Score increment if the solution improves the last admissible one but is not a best.
        'q3': 1.0,               # Score increment if the solution is accepted but does not improve the last admissible.
        'lrfi_count': 10,         # Maximum number of nodes to track for "Last Removed First Inserted" strategy.
        'ratio_sequence_to_remove': 0.15 # Random size of the sequence to remove
    }

    # Initialize HALNS heuristic with the problem instance file and parameters
    halns = HALNS(file_path, parameters)

    # Run HALNS to obtain the best solution
    best_solution = halns.run()

    # Output the best solution
    print("Best Solution:", best_solution)

    total_prize = halns.evaluate_solution(best_solution)
    print("Total Prize Collected:", total_prize)

Set parameter LazyConstraints to value 1


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 59 rows, 900 columns and 2608 nonzeros
Model fingerprint: 0x75619038
Variable types: 0 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [4e-01, 2e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 0 rows and 82 columns
Presolve time: 0.05s
Presolved: 59 rows, 818 columns, 2366 nonzeros
Variable types: 0 continuous, 818 integer (818 binary)

Root relaxation: objective 2.367258e+02, 120 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [13]:
def process_all_instances(directory_path):
    """
    Process all instance files in a directory using the HALNS metaheuristic,
    and record results in a CSV file. 

    :param directory_path: Path to the directory containing instance files.
    :return: A list of dictionaries, each containing the results for one instance.
    """
    import csv
    import os
    import time

    # Ensure you import your HALNS class. For example:
    # from src.halns import HALNS

    # Prepare a list to store the results
    results = []

    # Name of the CSV file for storing aggregated results
    csv_file = 'results_halns.csv'
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write the header row
        writer.writerow(['Instance', 'RunTime', 'Gap', 'TotalCollectedPrize', 'AveragePrizePerVehicle'])

    # Iterate over all files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):  # Adjust the extension if needed
            file_path = os.path.join(directory_path, filename)
            print(f"\nProcessing instance: {filename}")
            # ---------------------------------------------
            # 2. Initialize the HALNS metaheuristic
            # ---------------------------------------------
            
            params = {
                'T0': 95,              # Initial temperature for the simulated annealing mechanism.
                'T_min': 0.0001,              # Minimum temperature below which we reset temperature and solve a SPP.
                'cooling_rate': 0.9999,    # The factor by which the temperature is multiplied after each iteration.
                'Nseg': 100,              # Number of segments (run segments) after which score updates occur.
                'iteration_max': 1000,  # Maximum number of iterations per segment.
                'iteration_best_max': 50,# Maximum number of consecutive iterations without improvement before stopping.
                'kappa': 0.85,            # Reaction factor for updating operator/strategy scores.
                'q1': 20.0,              # Score increment if a new best solution is found.
                'q2': 5.0,               # Score increment if the solution improves the last admissible one but is not a best.
                'q3': 1.0,               # Score increment if the solution is accepted but does not improve the last admissible.
                'lrfi_count': 10,         # Maximum number of nodes to track for "Last Removed First Inserted" strategy.
                'ratio_sequence_to_remove': 0.15 # Random size of the sequence to remove
            }

            # Start timing
            start_time = time.time()

            # Create HALNS object
            halns_solver = HALNS(file_path=file_path, parameters=params)

            # ---------------------------------------------
            # 3. Run the HALNS metaheuristic
            # ---------------------------------------------
            best_solution = halns_solver.run()

            # End timing
            end_time = time.time()
            run_time = end_time - start_time

            # ---------------------------------------------
            # 4. Compute metrics: total collected prize, average prize per vehicle
            # ---------------------------------------------
            
            total_collected_prize = halns_solver.evaluate_solution(best_solution)

            # The number of vehicles is typically halns_solver.problem.P
            # or directly from the best_solution if it is a list of routes:
            # num_vehicles_used = len(best_solution)
            P = halns_solver.problem.P  # or from your problem data
            if P > 0:
                average_prize = total_collected_prize / P
            else:
                average_prize = 0

            # Gap is not directly computed by HALNS (unlike a MILP solver),
            # so we can store a placeholder value (e.g., 0 or None).
            gap = 0.0

            # ---------------------------------------------
            # 5. Append the results
            # ---------------------------------------------
            results.append({
                'Instance': filename,
                'RunTime': run_time,
                'Gap': gap,
                'TotalCollectedPrize': total_collected_prize,
                'AveragePrizePerVehicle': average_prize
            })

            # Write the results to the CSV file
            with open(csv_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([
                    filename,
                    run_time,
                    gap,
                    total_collected_prize,
                    average_prize
                ])

        else:
            print(f"Skipping non-txt file: {filename}")

    return results


In [14]:
logging.basicConfig(
    level=logging.WARNING,  # INFO / WORNING
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
instance_directory = 'Set_100_234'  
results = process_all_instances(instance_directory)
for result in results:
    print(result)


Processing instance: p4.2.a.txt
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 59 rows, 900 columns and 2608 nonzeros
Model fingerprint: 0x6365a4de
Variable types: 0 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [4e-01, 2e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 0 rows and 77 columns
Presolve time: 0.04s
Presolved: 59 rows, 823 columns, 2382 nonzeros
Variable types: 0 continuous, 823 integer (823 binary)

Root relaxation: objective 2.352704e+02, 100 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 E


Processing instance: p4.2.d.txt
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 161 rows, 6561 columns and 19438 nonzeros
Model fingerprint: 0x1065f046
Variable types: 0 continuous, 6561 integer (6561 binary)
Coefficient statistics:
  Matrix range     [2e-01, 3e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective -0.0000000
Presolve time: 0.05s
Presolved: 161 rows, 6561 columns, 19436 nonzeros
Variable types: 0 continuous, 6561 integer (6561 binary)

Root relaxation: objective 4.898902e+02, 284 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI


Processing instance: p4.2.e.txt
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 161 rows, 6561 columns and 19438 nonzeros
Model fingerprint: 0x6723acd5
Variable types: 0 continuous, 6561 integer (6561 binary)
Coefficient statistics:
  Matrix range     [2e-01, 4e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective -0.0000000
Presolve time: 0.04s
Presolved: 161 rows, 6561 columns, 19436 nonzeros
Variable types: 0 continuous, 6561 integer (6561 binary)

Root relaxation: objective 5.769441e+02, 271 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI


Processing instance: p4.2.q.txt
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 131 rows, 4356 columns and 12868 nonzeros
Model fingerprint: 0x85f4b33f
Variable types: 0 continuous, 4356 integer (4356 binary)
Coefficient statistics:
  Matrix range     [4e-01, 4e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve time: 0.02s
Presolved: 131 rows, 4356 columns, 12866 nonzeros
Variable types: 0 continuous, 4356 integer (4356 binary)

Root relaxation: objective 6.855122e+02, 271 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI


Processing instance: p4.2.r.txt
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 129 rows, 4225 columns and 12478 nonzeros
Model fingerprint: 0x179caf11
Variable types: 0 continuous, 4225 integer (4225 binary)
Coefficient statistics:
  Matrix range     [4e-01, 4e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve time: 0.02s
Presolved: 129 rows, 4225 columns, 12476 nonzeros
Variable types: 0 continuous, 4225 integer (4225 binary)

Root relaxation: objective 6.890442e+02, 304 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI

...


Processing instance: p4.4.e.txt
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11 rows, 36 columns and 88 nonzeros
Model fingerprint: 0xb39f0653
Variable types: 0 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 2 rows and 12 columns
Presolve time: 0.00s
Presolved: 9 rows, 24 columns, 57 nonzeros
Variable types: 0 continuous, 24 integer (24 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 4.400000e+01, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Ob


Processing instance: p4.4.l.txt
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 123 rows, 3844 columns and 11344 nonzeros
Model fingerprint: 0xe67713a4
Variable types: 0 continuous, 3844 integer (3844 binary)
Coefficient statistics:
  Matrix range     [2e-01, 3e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective -0.0000000
Presolve time: 0.02s
Presolved: 123 rows, 3844 columns, 11342 nonzeros
Variable types: 0 continuous, 3844 integer (3844 binary)

Root relaxation: objective 4.019911e+02, 211 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI


Processing instance: p4.4.n.txt
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 137 rows, 4761 columns and 14074 nonzeros
Model fingerprint: 0xdaed7d5a
Variable types: 0 continuous, 4761 integer (4761 binary)
Coefficient statistics:
  Matrix range     [2e-01, 4e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective -0.0000000
Presolve time: 0.02s
Presolved: 137 rows, 4761 columns, 14072 nonzeros
Variable types: 0 continuous, 4761 integer (4761 binary)

Root relaxation: objective 5.330891e+02, 224 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI


Processing instance: p4.4.r.txt
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 103 rows, 2704 columns and 7954 nonzeros
Model fingerprint: 0x9d35950d
Variable types: 0 continuous, 2704 integer (2704 binary)
Coefficient statistics:
  Matrix range     [2e-01, 4e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective -0.0000000
Presolve time: 0.02s
Presolved: 103 rows, 2704 columns, 7952 nonzeros
Variable types: 0 continuous, 2704 integer (2704 binary)

Root relaxation: objective 4.909237e+02, 191 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf

{'Instance': 'p4.2.a.txt', 'RunTime': 6.896641492843628, 'Gap': 0.0, 'TotalCollectedPrize': 206.0, 'AveragePrizePerVehicle': 103.0}
{'Instance': 'p4.2.b.txt', 'RunTime': 101.57292461395264, 'Gap': 0.0, 'TotalCollectedPrize': 341.0, 'AveragePrizePerVehicle': 170.5}
{'Instance': 'p4.2.c.txt', 'RunTime': 147.65275359153748, 'Gap': 0.0, 'TotalCollectedPrize': 0.0, 'AveragePrizePerVehicle': 0.0}
{'Instance': 'p4.2.d.txt', 'RunTime': 223.83649730682373, 'Gap': 0.0, 'TotalCollectedPrize': 0.0, 'AveragePrizePerVehicle': 0.0}
{'Instance': 'p4.2.e.txt', 'RunTime': 233.86602401733398, 'Gap': 0.0, 'TotalCollectedPrize': 606.0, 'AveragePrizePerVehicle': 303.0}
{'Instance': 'p4.2.f.txt', 'RunTime': 677.1125829219818, 'Gap': 0.0, 'TotalCollectedPrize': 669.0, 'AveragePrizePerVehicle': 334.5}
{'Instance': 'p4.2.g.txt', 'RunTime': 133.95486879348755, 'Gap': 0.0, 'TotalCollectedPrize': 719.0, 'AveragePrizePerVehicle': 359.5}
{'Instance': 'p4.2.h.txt', 'RunTime': 198.19384932518005, 'Gap': 0.0, 'TotalCol